# 🧬 Medical AI Virtual Staining - Experiment Training

This notebook runs training experiments on Google Colab GPU.

## Setup
1. **Runtime** → **Change runtime type** → **T4 GPU**
2. Set the `EXPERIMENT_BRANCH` variable below
3. Run all cells

In [ ]:
# ============================================================
# Configuration - CHANGE THIS FOR EACH EXPERIMENT
# ============================================================

EXPERIMENT_BRANCH = "experiment/exp_002_augmentation"  # Change this!
# Options:
# - "experiment/exp_002_augmentation"
# - "experiment/exp_003_l1_only"
# - "experiment/exp_004_larger_batch"
# - "experiment/exp_005_image_size_256"
# - "experiment/exp_006_ssim_weight"
# - "main" (baseline)

EPOCHS = 3  # Number of training epochs
REPO_URL = "https://github.com/osukee/7th-National-Medical-AI-Competition.git"

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository and checkout experiment branch
!git clone {REPO_URL} repo
%cd repo
!git checkout {EXPERIMENT_BRANCH}
!git log -1 --oneline

In [ ]:
# Install dependencies
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q segmentation-models-pytorch scikit-image pandas tqdm pillow

In [ ]:
# Setup Kaggle API for data download
# You need to upload your kaggle.json file
from google.colab import files
print("Please upload your kaggle.json file:")
uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download competition data
!pip install -q kaggle
!kaggle competitions download -c medical-ai-contest-7th-2025
!unzip -q medical-ai-contest-7th-2025.zip -d /content/data
!ls /content/data

In [ ]:
# Modify train_notebook.py to use Colab paths
import os

# Read the training script
with open('kaggle/train_notebook.py', 'r') as f:
    content = f.read()

# Replace Kaggle paths with Colab paths
content = content.replace(
    'data_dir = Path("/kaggle/input/medical-ai-contest-7th-2025")',
    'data_dir = Path("/content/data")'
)
content = content.replace(
    'output_dir = Path("/kaggle/working")',
    'output_dir = Path("/content/output")'
)

# Write modified script
with open('kaggle/train_notebook.py', 'w') as f:
    f.write(content)

# Create output directory
os.makedirs('/content/output', exist_ok=True)

print("Paths updated for Colab!")

In [ ]:
# Set environment variables
import os
os.environ['EPOCHS'] = str(EPOCHS)
os.environ['EXPERIMENT_ID'] = EXPERIMENT_BRANCH.split('/')[-1]
os.environ['BRANCH_NAME'] = EXPERIMENT_BRANCH

print(f"Running experiment: {os.environ['EXPERIMENT_ID']}")
print(f"Epochs: {os.environ['EPOCHS']}")

In [ ]:
# 🚀 Run Training!
%cd /content/repo
!python kaggle/train_notebook.py

In [ ]:
# Display Results
import json

with open('/content/output/metrics.json', 'r') as f:
    metrics = json.load(f)

print("="*50)
print(f"🎯 EXPERIMENT RESULTS: {metrics['experiment_id']}")
print("="*50)
print(f"SSIM: {metrics['metrics']['ssim']:.4f}")
print(f"PSNR: {metrics['metrics']['psnr']:.2f}")
print(f"Training Time: {metrics['training_time_seconds']/60:.1f} minutes")
print("="*50)

# Show config
print("\nConfig:")
for key, value in metrics['config'].items():
    print(f"  {key}: {value}")

In [ ]:
# Download results
from google.colab import files
files.download('/content/output/metrics.json')
files.download('/content/output/best_model.pth')